---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
print(df.iloc[14])

#print(df)
print(df.str.findall(r'\d+/\d+(?:/\d+)?|(?:\d+ )?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[.,]?(?:-\d+-\d+| \d+(?:th|rd|st|nd)?,? \d+|\d+)|[1-2]\d{3}')  ) 
x=df.str.findall(r'\d+/\d+(?:/\d+)?|(?:\d+ )?(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[.,]?(?:-\d+-\d+| \d+(?:th|rd|st|nd)?,? \d+|\d+)|[1-2]\d{3}') 
print(x.tail(50))
#print(df.str.findall(r'\d+?-?/?\d+?-?/?\d+'))
#print(df.str.findall(r'[ ][0-2]\d{3}'))
#s=df.str.findall(r'/d+[/][0-2]\d\d\d')
#for i in s:
#    if len(i)>0:
#        print(i)
#print(df.str.findall(r'/d+[/][0-2]\d\d\d'))


5/24/1990 CPT Code: 90792: With medical services

0            [03/25/93]
1             [6/18/85]
2              [7/8/71]
3             [9/27/75]
4              [2/6/96]
5             [7/06/79]
6             [5/18/78]
7            [10/24/89]
8              [3/7/86]
9             [4/10/71]
10     [5/11/85, 16/22]
11            [4/09/75]
12            [8/01/98]
13            [1/26/72]
14          [5/24/1990]
15          [1/25/2011]
16            [4/12/82]
17         [10/13/1976]
18            [4/24/98]
19            [5/21/77]
20            [7/21/98]
21           [10/21/79]
22            [3/03/90]
23            [2/11/76]
24         [07/25/1984]
25                   []
26            [9/22/89]
27            [9/02/76]
28            [9/12/71]
29           [10/24/86]
             ...       
470              [1983]
471              [1999]
472              [2010]
473              [1975]
474              [1972]
475              [2015]
476              [1989]
477              [1994]
478           

In [2]:
def date_sorter():
    
    df_ans = df.str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4})')
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])
    df_ans = df_ans.append(df[not_duplicated].str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2})'))
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])
    df_ans = df_ans.append(df[not_duplicated].str.extractall(r'(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4})'))
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])
    df_ans = df_ans.append(df[not_duplicated].str.extractall(r'(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4})'))
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])

    without_day = df[not_duplicated].str.extractall('(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4})')
    without_day = without_day.append(df[not_duplicated].str.extractall(r'(?P<month>\d\d?)/(?P<year>\d{4})'))
    without_day['day'] = 1
    df_ans = df_ans.append(without_day)
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])

    without_month = df[not_duplicated].str.extractall(r'(?P<year>\d{4})')
    without_month['day'] = 1
    without_month['month'] = 1
    df_ans = df_ans.append(without_month)
    not_duplicated = ~df.index.isin([index[0] for index in df_ans.index])

    # Year
    df_ans['year'] = df_ans['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    df_ans['year'] = df_ans['year'].apply(lambda x: str(x))

    # Month
    df_ans['month'] = df_ans['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    month_dict = {'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1,'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3, 'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8}
    
    df_ans = df_ans.replace(month_dict)
    df_ans['month'] = df_ans['month'].apply(lambda x: str(x))
    df_ans['day'] = df_ans['day'].apply(lambda x: str(x))

    df_ans['date'] = df_ans['month'] + '/' + df_ans['day'] + '/' + df_ans['year']
    df_ans['date'] = pd.to_datetime(df_ans['date'])

    df_ans = df_ans.sort_values(by='date')
    return_rank = pd.Series(list(df_ans.index.labels[0]))
    
    return return_rank 
    
    

In [3]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    286
477    244
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64